<a href="https://colab.research.google.com/github/roscoekerby/google-colab/blob/main/Dynamic_Podcasts_Episodes_Scraper_2025_02_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Imports

In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 27.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Pack

In [3]:
!pip install webdriver-manager

# **Scrape Apple 2024-10-28**

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1920, 1080)  # Set a larger window size
    return driver

def wait_for_episodes(driver, current_count):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: len(x.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")) > current_count
        )
        return True
    except TimeoutException:
        return False

def scroll_to_bottom(driver):
    previous_episode_count = 0
    no_new_content_count = 0
    max_attempts = 10  # Increased max attempts

    while no_new_content_count < max_attempts:
        # Get current episode count
        current_episodes = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        current_count = len(current_episodes)

        if current_count == previous_episode_count:
            no_new_content_count += 1
        else:
            no_new_content_count = 0

        # Scroll in smaller increments
        if current_episodes:
            for i in range(max(0, len(current_episodes)-5), len(current_episodes)):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", current_episodes[i])
                    time.sleep(0.5)
                except:
                    continue

        time.sleep(2)

        new_content_loaded = wait_for_episodes(driver, current_count)
        if not new_content_loaded:
            no_new_content_count += 1

        previous_episode_count = current_count
        print(f"Current episode count: {current_count}")

def extract_episode_info(episode, index):
    """Extract information from an episode element with enhanced retry logic."""
    max_retries = 5  # Increased retries
    for attempt in range(max_retries):
        try:
            # Wait for elements to be present
            WebDriverWait(episode, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.episode-details__title-text"))
            )

            title = episode.find_element(By.CSS_SELECTOR, "span.episode-details__title-text").text
            link = episode.find_element(By.CSS_SELECTOR, "a.link-action").get_attribute('href')
            description = episode.find_element(By.CSS_SELECTOR, "p.episode-details__summary").text

            if not all([title, link, description]):
                raise NoSuchElementException("Missing data")

            return {
                'index': index,
                'title': title,
                'link': link,
                'description': description
            }
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if attempt == max_retries - 1:
                print(f"Failed to extract episode at index {index} after {max_retries} attempts: {str(e)}")
            time.sleep(1)
            continue
    return None

def scrape_episodes(url):
    episode_data = []
    driver = init_driver()

    try:
        driver.get(url)

        # Wait for initial load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.svelte-8rlk6b"))
        )

        # Scroll to load all episodes
        scroll_to_bottom(driver)

        # Get all episodes after scrolling
        episode_elements = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        print(f"Total episodes found: {len(episode_elements)}")

        # Extract information from each episode
        for index, episode in enumerate(episode_elements):
            episode_info = extract_episode_info(episode, index)
            if episode_info:
                episode_data.append(episode_info)
                print(f"Scraped ({index + 1}/{len(episode_elements)}): {episode_info['title']}")
            else:
                print(f"Failed to scrape episode at index {index}")

        # Sort episodes by index to maintain order
        episode_data.sort(key=lambda x: x['index'])

        # Check for missing episodes
        titles = [ep['title'] for ep in episode_data]
        for i in range(1, 27):  # Adjust range based on expected episode count
            episode_num = f"TMGP Ep {str(i).zfill(2)}"
            if not any(episode_num in title for title in titles):
                print(f"WARNING: Potentially missing {episode_num}")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

    return episode_data

if __name__ == "__main__":
    url = "https://podcasts.apple.com/za/podcast/the-muscle-growth-podcast/id1717906577/episodes"
    apple_episodes = scrape_episodes(url)
    print(f"\nTotal episodes successfully scraped: {len(apple_episodes)}")

    # Print all episode titles in order
    print("\nAll scraped episodes:")
    for ep in apple_episodes:
        print('*' * 80)
        print(f"{ep['title']}")
        print(f"{ep['link']}")
        print(f"{ep['description']}")
        print('*' * 80)


Current episode count: 25
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Total episodes found: 38
Failed to extract episode at index 0 after 5 attempts: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p.episode-details__summary"}
  (Session info: chrome=135.0.7049.95); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5c9ff4cbbcea <unknown>
#1 0x5c9ff476c5f0 <unknown>
#2 0x5c9ff47bda33 <unknown>
#3 0x5c9ff47bdc21 <unknown>
#4 0x5c9ff47b15c6 <unknown>
#5 0x5c9ff47e368d <unknown>
#6 0x5c9ff47b14ba <unknown>
#7 0x5c9ff47e382e <unknown>
#8 0x5c9ff4809660 <unknown>
#9 0x5c9ff47e3433 <unknown>
#10 0x5c9ff47afea3 <unknown>
#11 0x5c9ff47b0b01 <unknown>
#12 0x5c9ff4c80b3b <unknown>
#13 0x5c9ff4c84a21 <unknown>
#14 0x5c9ff4c67c32 <unknown

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1920, 1080)  # Set a larger window size
    return driver

def wait_for_episodes(driver, current_count):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: len(x.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")) > current_count
        )
        return True
    except TimeoutException:
        return False

def scroll_to_bottom(driver):
    previous_episode_count = 0
    no_new_content_count = 0
    max_attempts = 10  # Increased max attempts

    while no_new_content_count < max_attempts:
        # Get current episode count
        current_episodes = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        current_count = len(current_episodes)

        if current_count == previous_episode_count:
            no_new_content_count += 1
        else:
            no_new_content_count = 0

        # Scroll in smaller increments
        if current_episodes:
            for i in range(max(0, len(current_episodes)-5), len(current_episodes)):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", current_episodes[i])
                    time.sleep(0.5)
                except:
                    continue

        time.sleep(2)

        new_content_loaded = wait_for_episodes(driver, current_count)
        if not new_content_loaded:
            no_new_content_count += 1

        previous_episode_count = current_count
        print(f"Current episode count: {current_count}")

def extract_episode_info(episode, index):
    """Extract episode information with improved error handling."""
    max_retries = 5
    for attempt in range(max_retries):
        try:
            # Wait for elements
            WebDriverWait(episode, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='episode-lockup-title']"))
            )

            title = episode.find_element(By.CSS_SELECTOR, "[data-testid='episode-lockup-title']").text
            link = episode.find_element(By.CSS_SELECTOR, "[data-testid='click-action']").get_attribute('href')

            try:
                description = episode.find_element(By.CSS_SELECTOR, "[data-testid='episode-content__summary']").text
            except NoSuchElementException:
                description = "No description available"

            if not title or not link:
                raise NoSuchElementException("Missing title or link")

            return {'index': index, 'title': title, 'link': link, 'description': description}
        except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"Retry {attempt + 1}/{max_retries}: Failed to extract episode at index {index} - {e}")
            time.sleep(1)
            continue
    return None


def scrape_episodes(url):
    episode_data = []
    driver = init_driver()

    try:
        driver.get(url)

        # Wait for initial load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.svelte-8rlk6b"))
        )

        # Scroll to load all episodes
        scroll_to_bottom(driver)

        # Get all episodes after scrolling
        episode_elements = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        print(f"Total episodes found: {len(episode_elements)}")

        # Extract information from each episode
        for index, episode in enumerate(episode_elements):
            episode_info = extract_episode_info(episode, index)
            if episode_info:
                episode_data.append(episode_info)
                print(f"Scraped ({index + 1}/{len(episode_elements)}): {episode_info['title']}")
            else:
                print(f"Failed to scrape episode at index {index}")

        # Sort episodes by index to maintain order
        episode_data.sort(key=lambda x: x['index'])

        # Check for missing episodes
        titles = [ep['title'] for ep in episode_data]
        for i in range(1, 27):  # Adjust range based on expected episode count
            episode_num = f"TMGP Ep {str(i).zfill(2)}"
            if not any(episode_num in title for title in titles):
                print(f"WARNING: Potentially missing {episode_num}")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

    return episode_data

if __name__ == "__main__":
    url = "https://podcasts.apple.com/za/podcast/the-muscle-growth-podcast/id1717906577/episodes"
    apple_episodes = scrape_episodes(url)
    print(f"\nTotal episodes successfully scraped: {len(apple_episodes)}")

    # Print all episode titles in order
    print("\nAll scraped episodes:")
    for ep in apple_episodes:
        print('*' * 80)
        print(f"{ep['title']}")
        print(f"{ep['link']}")
        print(f"{ep['description']}")
        print('*' * 80)


Current episode count: 25
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Total episodes found: 38
Scraped (1/38): TMGP Ep 38 with NO-BS strength and fitness coach Cillian O’Connor AKA DYSFUNCTIONALPATTERNS
Scraped (2/38): TMGP Ep 37 with men’s physique winner, bodybuilder, and lifestyle coach - Jonathan Kantor
Scraped (3/38): TMGP Ep 36 with Emanuel Pescari: Austria’s Strongest Man, MULTIPLE WORLD RECORD TITLE HOLDER & INTERNATIONAL STRONGMAN SUPERSTAR Competitor part 2
Scraped (4/38): TMGP Ep 35 with Emanuel Pescari: Austria’s Strongest Man and Elite Strongman Competitor part 1
Scraped (5/38): TMGP Ep 34 with health coach, wellness consultant mindset expert Jeffrey Siegel part 2
Scraped (6/38): TMGP Ep 33 with health coach, wellness consultant, and mindset expert Jeffrey Siegel
Scraped (7/38): TMGP Ep 32 with health coach and diabetes expert Ghamdan Al-Areeky
Scraped (8/38): TM

# **Scrape Spotify**

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

# Function to initialize the WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to click elements using CSS selectors with a retry mechanism
def click_element(driver, selector, max_retries=5, wait_time=3):
    retries = 0
    while retries < max_retries:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            driver.execute_script("arguments[0].scrollIntoView();", element)  # Ensure button is visible
            element.click()
            time.sleep(wait_time)  # Add a wait time to load additional episodes
            return True
        except Exception as e:
            retries += 1
            print(f"Retry {retries}/{max_retries} - Failed to click: {e}")
            time.sleep(wait_time)
    print(f"Failed to click on element after {max_retries} retries.")
    return False

# Function to load all episodes by clicking "Load more episodes" until it's unavailable
def load_all_episodes(driver):
    load_more_selector = 'button[data-encore-id="chip"][role="checkbox"]'
    while click_element(driver, load_more_selector):
        print("Clicked 'Load more episodes' button.")
        time.sleep(3)  # Delay for loading new episodes

# Function to extract episode links from the infinite scroll list
def get_episode_links(driver):
    episode_and_show_list = []
    infinite_scroll_list = driver.find_element(By.CSS_SELECTOR, '[data-testid="infinite-scroll-list"]')
    episode_links = infinite_scroll_list.find_elements(By.TAG_NAME, 'a')

    for episode_link in episode_links:
        episode_href = episode_link.get_attribute('href')
        if episode_href and 'show' not in episode_href:
            print("Episode Link:", episode_href)
            episode_and_show_list.append(episode_href)
    return episode_and_show_list

# Main execution
if __name__ == "__main__":
    url = "https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV"
    driver = init_driver()
    driver.get(url)

    try:
        # Load all episodes by clicking "Load more episodes" until no more episodes can be loaded
        load_all_episodes(driver)

        # Extract and print episode links
        spotify_episodes = get_episode_links(driver)
        print("Total Episodes Found:", len(spotify_episodes))

    finally:
        driver.quit()


Retry 1/5 - Failed to click: Message: 
Stacktrace:
#0 0x55c75178fcea <unknown>
#1 0x55c7512405f0 <unknown>
#2 0x55c751291a33 <unknown>
#3 0x55c751291c21 <unknown>
#4 0x55c7512e0274 <unknown>
#5 0x55c7512b768d <unknown>
#6 0x55c7512dd660 <unknown>
#7 0x55c7512b7433 <unknown>
#8 0x55c751283ea3 <unknown>
#9 0x55c751284b01 <unknown>
#10 0x55c751754b3b <unknown>
#11 0x55c751758a21 <unknown>
#12 0x55c75173bc32 <unknown>
#13 0x55c751759594 <unknown>
#14 0x55c75171feef <unknown>
#15 0x55c75177dd98 <unknown>
#16 0x55c75177df76 <unknown>
#17 0x55c75178eb36 <unknown>
#18 0x7d0d22479ac3 <unknown>

Retry 2/5 - Failed to click: Message: 
Stacktrace:
#0 0x55c75178fcea <unknown>
#1 0x55c7512405f0 <unknown>
#2 0x55c751291a33 <unknown>
#3 0x55c751291c21 <unknown>
#4 0x55c7512e0274 <unknown>
#5 0x55c7512b768d <unknown>
#6 0x55c7512dd660 <unknown>
#7 0x55c7512b7433 <unknown>
#8 0x55c751283ea3 <unknown>
#9 0x55c751284b01 <unknown>
#10 0x55c751754b3b <unknown>
#11 0x55c751758a21 <unknown>
#12 0x55c75173bc32

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-testid="infinite-scroll-list"]"}
  (Session info: chrome=135.0.7049.95); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55c75178fcea <unknown>
#1 0x55c7512405f0 <unknown>
#2 0x55c751291a33 <unknown>
#3 0x55c751291c21 <unknown>
#4 0x55c7512e0274 <unknown>
#5 0x55c7512b768d <unknown>
#6 0x55c7512dd660 <unknown>
#7 0x55c7512b7433 <unknown>
#8 0x55c751283ea3 <unknown>
#9 0x55c751284b01 <unknown>
#10 0x55c751754b3b <unknown>
#11 0x55c751758a21 <unknown>
#12 0x55c75173bc32 <unknown>
#13 0x55c751759594 <unknown>
#14 0x55c75171feef <unknown>
#15 0x55c75177dd98 <unknown>
#16 0x55c75177df76 <unknown>
#17 0x55c75178eb36 <unknown>
#18 0x7d0d22479ac3 <unknown>


In [ ]:
spotify_episodes

In [ ]:
spotify_episode_urls = []
for url in spotify_episodes:
  if "episode" in url:
    print(url)
    spotify_episode_urls.append(url)

# Concactenate Info from Apple and Spotify

In [ ]:
def create_episode_info(apple_episodes, spotify_episode_urls):
    episodes = []

    # Ensure matching lengths between apple episodes and spotify urls
    if len(apple_episodes) != len(spotify_episode_urls):
        print("Warning: Mismatched number of Apple episodes and Spotify URLs.")
        return []

    for i in range(len(apple_episodes)):
        episode_info = {
            "title": apple_episodes[i]["title"],
            "description": apple_episodes[i]["description"],
            "urls": {
                "spotify": spotify_episode_urls[i],
                "apple": apple_episodes[i]["link"]
            }
        }
        episodes.append(episode_info)

    return episodes

episodes = create_episode_info(apple_episodes, spotify_episode_urls)


# Format into HTML for WEBSITE

In [ ]:
def print_formatted_episodes_html(episodes):
    html_output = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Episodes</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            .episode-card {
                border: 1px solid #ddd;
                padding: 20px;
                margin: 10px 0;
                border-radius: 8px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }
            .episode-title {
                font-size: 1.5em;
                font-weight: bold;
                margin-bottom: 10px;
            }
            .episode-subtitle {
                font-size: 1.2em;
                font-weight: bold;
                color: #333;
                margin: 5px 0;
            }
            .episode-description {
                font-size: 1em;
                line-height: 1.6;
                margin: 10px 0;
            }
            .platform-link {
                font-weight: bold;
                text-decoration: none;
                margin-right: 10px;
            }
            .apple-link {
                color: #0073e6;
            }
            .spotify-link {
                color: #1DB954;
            }
        </style>
    </head>
    <body>
    <div id="episodes-container">
    """

    # Generate HTML for each episode in reverse order
    for i, episode in enumerate(reversed(episodes), 1):  # Latest to oldest order
        title = episode["title"]
        description = episode["description"]
        apple_url = episode["urls"]["apple"]
        spotify_url = episode["urls"]["spotify"]

        html_output += f"""
        <div class="episode-card">
            <div class="episode-title">Episode {i}</div>
            <div class="episode-subtitle">{title}</div>
            <div class="episode-description">{description}</div>
            <p>
                <a href="{apple_url}" target="_blank" class="platform-link apple-link">
                    🍏 Listen on Apple
                </a>
                <a href="{spotify_url}" target="_blank" class="platform-link spotify-link">
                    🎶 Listen on Spotify
                </a>
            </p>
        </div>
        """

    # Close the main container and body tags
    html_output += """
    </div>
    </body>
    </html>
    """

    print(html_output)

# Example usage:
episodes = create_episode_info(apple_episodes, spotify_episode_urls)
print_formatted_episodes_html(episodes)
